In [1]:
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

sys.path.append('..')

from run_etl import run_all

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("SETUP UKOŃCZONY")
print(f"Data uruchomienia: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Wersja Python: {sys.version}")
print(f"Wersja Pandas: {pd.__version__}")

SETUP UKOŃCZONY
Data uruchomienia: 2025-08-20 19:25:59
Wersja Python: 3.13.6 (tags/v3.13.6:4e66535, Aug  6 2025, 14:36:00) [MSC v.1944 64 bit (AMD64)]
Wersja Pandas: 2.3.1


In [2]:
print("URUCHAMIANIE ETL PIPELINE...")
print("=" * 60)

INPUT_PATH = "../data/raw/online_retail_II.xlsx"
OUTPUT_PATH = "../data/processed"

try:
    etl_results = run_all(
        input_excel_path=INPUT_PATH,
        output_dir=OUTPUT_PATH,
        date_buffer_months=6
    )

    print("ETL PIPELINE UKOŃCZONYH POMYŚLNIE!")

except Exception as e:
    print(f"BŁĄD ETL PIPELINE: {e}")
    raise

URUCHAMIANIE ETL PIPELINE...
[2025-08-20 19:25:59] === START: RUN ALL ETL ===
[2025-08-20 19:25:59] Ładowanie surowych danych z: ../data/raw/online_retail_II.xlsx
[2025-08-20 19:26:59] Wczytano 1,067,371 rekordów z 2 arkuszy
[2025-08-20 19:26:59] Walidacja obecnosci kolumn RAW...
[2025-08-20 19:26:59] Konwersja typów (Quantity, Price, InvoiceDate)...
[2025-08-20 19:27:01] Deduplikacja rekordów...
[2025-08-20 19:27:01] Usunięto duplikatów: 34,337
[2025-08-20 19:27:01] Filtrowanie prawidłowych sprzedaży...
[2025-08-20 19:27:01] Po filtrowaniu: 1,007,912 rekordów
[2025-08-20 19:27:01] Standaryzacja krajów...
[2025-08-20 19:27:01] Czyszczenie opisów produktu...
[2025-08-20 19:27:02] Kalkulacja Total_Value...
[2025-08-20 19:27:02] Łączny przychód: £20,476,082.17
[2025-08-20 19:27:02] Quality Gates po czyszczeniu...
[2025-08-20 19:27:02] Budowa Dim_Geography...
[2025-08-20 19:27:02] Budowa Dim_Customer...
[2025-08-20 19:27:02] Budowa Dim_Product...
[2025-08-20 19:27:02] Budowa Dim_Date...
[2

In [3]:
print("ANALIZA WYMIARÓW MODELU GWIAZDY")
print("=" * 60)

dim_customer = etl_results['dim_customer']
dim_product = etl_results['dim_product']
dim_geography = etl_results['dim_geography']
dim_date = etl_results['dim_date']
fact_sales = etl_results['fact_sales']

print(f"Dim_Customer: {len(dim_customer):,} wierszy")
print(f"Dim_Product: {len(dim_product):,} wierszy")
print(f"Dim_Geography: {len(dim_geography):,} wierszy")
print(f"Dim_Date: {len(dim_date):,} wierszy")
print(f"Fact_Sales: {len(fact_sales):,} wierszy")

print("\nSzczegóły wymiarów:")
print(f"Klienci zarejestrowani: {len(dim_customer[dim_customer['CustomerType'] == 'Registered']):,}")
print(f"Klienci goście: {len(dim_customer[dim_customer['CustomerType'] == 'Guest']):,}")
print(f"Produkty prezentowe: {dim_product['IsGift'].sum():,}")
print(f"Kraje UK: {dim_geography['IsUK'].sum():,}")
print(f"Zakres dat: {dim_date['Date'].min().date()} do {dim_date['Date'].max().date()}")


ANALIZA WYMIARÓW MODELU GWIAZDY
Dim_Customer: 5,879 wierszy
Dim_Product: 4,916 wierszy
Dim_Geography: 43 wierszy
Dim_Date: 1,105 wierszy
Fact_Sales: 1,007,912 wierszy

Szczegóły wymiarów:
Klienci zarejestrowani: 5,878
Klienci goście: 1
Produkty prezentowe: 0
Kraje UK: 1
Zakres dat: 2009-06-01 do 2012-06-09
